In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset, random_split
import math

# temporary
import matplotlib.pyplot as plt

# Deep Learning project

## Preprocessing

Clone the 'Hey Waldo' repository.

In [ ]:
!git clone https://github.com/vc1492a/Hey-Waldo.git

The first step is to load our images dataset. To do this, we define a transformation that is used on the images. Here we resize all of our images to 128x128. Then, we load the 64x64 and 128x128 images in and apply the transformation on every image. Lastly, we combine the 2 datasets.

In [ ]:
transform = transforms.Compose([transforms.Resize(128),
                                transforms.ToTensor()])

images_64 = datasets.ImageFolder('/content/Hey-Waldo/64', transform = transform)
images_128 = datasets.ImageFolder('/content/Hey-Waldo/128', transform = transform)
images = ConcatDataset([images_64, images_128])

Next, we split our images into a train, validation and test set. The distribution is 81% of our images are used for training, 9% for validation and the other 10% for testing.

In [ ]:
N_train = int(0.9 * len(images))
N_test = int(0.1 * len(images))
N_validation = int(math.ceil(N_train * 0.1))

train, test_set = random_split(images, [N_train, N_test])
train_set, validation_set = random_split(train, [int(N_train * 0.9), N_validation])

We calculate the mean and standard deviation of the training set so that we can use this to normalize our data. We also define some augmentations for our images.

In [ ]:
def calc_mean_std(data):
  images = [item[0] for item in data]
  mean = torch.mean(torch.stack(images, dim=0), dim=(0, 2, 3))
  std = torch.std(torch.stack(images, dim=0), dim=(0, 2, 3))
  return mean, std

mean, std = calc_mean_std(train_set)

train_transformer = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.Normalize(mean, std)]) # Add more augmentations depending on what we want
test_transformer = transforms.Compose([transforms.Normalize(mean, std)])

# Apply transformation on train, validation and test sets